<a href="https://colab.research.google.com/github/Euna-Kim322/769_NLP_HLE/blob/main/BERTweet_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!chmod +x /content/drive/MyDrive/ColabNotebooks/hw3/setup.sh  # Make the script executable
!./drive/MyDrive/ColabNotebooks/hw3/setup.sh  # Run the script

In [3]:
!apt-get update
!apt-get install -y build-essential
!pip install tokenizers
!pip install torch-optimizer

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [589 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,185 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,455 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,012 kB]
Get:1

In [16]:
!pip install transformers
!pip3 install emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.6 MB/s eta 0:00:00


In [28]:
!python /content/drive/MyDrive/ColabNotebooks/hw3/classifier.py \
    --use_gpu \
    --option finetune \
    --lr 1e-5 \
    --seed 1234 \
    --train "/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-train.csv" \
    --dev "/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-dev.csv" \
    --test "/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-test.csv" \
    --dev_out "/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-dev-out.txt" \
    --test_out "/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-test-out.txt" \
    --filepath "/content/drive/MyDrive/ColabNotebooks/hw3/data/$mass_shooting-model.pt" | tee /content/drive/MyDrive/ColabNotebooks/hw3/mess_shooting-train-log.txt
#lr, finetune vs pretrain mode, seed?, model(Nadam, RAdam), from pretrained(bertweet-base)

args: {'train': '/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-train.csv', 'dev': '/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-dev.csv', 'test': '/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-test.csv', 'seed': 1234, 'epochs': 10, 'option': 'finetune', 'use_gpu': True, 'dev_out': '/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-dev-out.txt', 'test_out': '/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-test-out.txt', 'filepath': '/content/drive/MyDrive/ColabNotebooks/hw3/data/-model.pt', 'batch_size': 8, 'hidden_dropout_prob': 0.3, 'lr': 1e-05}
load 2000 data from /content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-train.csv
load 500 data from /content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-dev.csv
save the model to /content/drive/MyDrive/ColabNotebooks/hw3/data/-model.pt
epoch 0: train loss :: -3.491, train acc :: 0.782, dev acc :: 0.796
save the model to /content/drive/MyDrive/ColabNotebooks

In [31]:
import pandas as pd
from sklearn.metrics import f1_score, classification_report

# Read the file and split by lines
with open("/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-test-out.txt", 'r') as file:
    lines = file.readlines()

# Parse the context, true label, and predicted label from each line
contexts, true_labels, predicted_labels = [], [], []

for line in lines:
    parts = line.strip().split(" ||| ")
    if len(parts) == 3:
        contexts.append(parts[0])
        true_labels.append(int(parts[1]))
        predicted_labels.append(int(parts[2]))

# Convert to DataFrame for easier analysis
df = pd.DataFrame({
    'Context': contexts,
    'True_Label': true_labels,
    'Predicted_Label': predicted_labels
})

# Display the first few rows
print(df.head())

# Basic Analysis
print("\nTotal number of records:", len(df))
print("\nNumber of correctly predicted labels:", sum(df['True_Label'] == df['Predicted_Label']))

# For a more detailed error analysis
errors = df[df['True_Label'] != df['Predicted_Label']]
print("\nNumber of incorrect predictions:", len(errors))

# Display first 10 incorrect predictions for inspection
print("\nSample Incorrect Predictions:")
print(errors.head(10))

# Calculate the F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')
print("\nF1 Score:", f1)

# Classification Report for more detailed metrics
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


                                             Context  True_Label  \
0  rt @ddlovato: really sad to hear that there wa...           3   
1  @coolneguy border patrol agent accused in fata...           0   
2  rt @goldwatergal: repeat after me: gun control...           0   
3  east palo alto: one killed, four injured in sh...           0   
4  rt @tweetslmao: so if guns kill people, i gues...           0   

   Predicted_Label  
0                3  
1                0  
2                0  
3                0  
4                0  

Total number of records: 1500

Number of correctly predicted labels: 1359

Number of incorrect predictions: 141

Sample Incorrect Predictions:
                                              Context  True_Label  \
14  rt @willspencer: i can't wait to be lectured a...           2   
33  rt @seamuskrat: i gave @molonlabenj +k about g...           2   
54  rt @theblaze: 163 dems sign letter to john boe...           0   
55  illinoiscarry on possible bill this week.

In [30]:

# Import necessary libraries
import pandas as pd

# Read the file and split by lines
with open("/content/drive/MyDrive/ColabNotebooks/hw3/data/mass_shooting-test-out.txt", 'r') as file:
    lines = file.readlines()

# Parse the context, true label, and predicted label from each line
contexts, true_labels, predicted_labels = [], [], []

for line in lines:
    parts = line.strip().split(" ||| ")
    if len(parts) == 3:
        contexts.append(parts[0])
        true_labels.append(int(parts[1]))
        predicted_labels.append(int(parts[2]))

# Convert to DataFrame for easier analysis
df = pd.DataFrame({
    'Context': contexts,
    'True_Label': true_labels,
    'Predicted_Label': predicted_labels
})

# Display the first few rows
print(df.head())

# Basic Analysis
print("\nTotal number of records:", len(df))
print("\nNumber of correctly predicted labels:", sum(df['True_Label'] == df['Predicted_Label']))

# For a more detailed error analysis
errors = df[df['True_Label'] != df['Predicted_Label']]
print("\nNumber of incorrect predictions:", len(errors))

# Display first few incorrect predictions for inspection
print("\nSample Incorrect Predictions:")
print(errors.head())


                                             Context  True_Label  \
0  rt @ddlovato: really sad to hear that there wa...           3   
1  @coolneguy border patrol agent accused in fata...           0   
2  rt @goldwatergal: repeat after me: gun control...           0   
3  east palo alto: one killed, four injured in sh...           0   
4  rt @tweetslmao: so if guns kill people, i gues...           0   

   Predicted_Label  
0                3  
1                0  
2                0  
3                0  
4                0  

Total number of records: 1500

Number of correctly predicted labels: 1359

Number of incorrect predictions: 141

Sample Incorrect Predictions:
                                              Context  True_Label  \
14  rt @willspencer: i can't wait to be lectured a...           2   
33  rt @seamuskrat: i gave @molonlabenj +k about g...           2   
54  rt @theblaze: 163 dems sign letter to john boe...           0   
55  illinoiscarry on possible bill this week.

In [31]:
!pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view

utils.logging.set_verbosity_error()  # Suppress standard warnings

model_name = "microsoft/xtremedistil-l12-h384-uncased"  # Find popular HuggingFace models here: https://huggingface.co/models
input_text = "The cat sat on the mat"
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
model_view(attention, tokens)  # Display model view